#Estimating the Test Error

We learned about _training error_, which is the error calculated on the training data. Training error is easy to calculate because the true labels are available in the training data. However, training error is not always a good measure of a model's quality, since a model that _overfits_ to the training data may have artificially low training error.

Ideally, we would like to evaluate regression models based on their _test error_, which is the error calculated on the test data. The problem with test error is that it is usually impossible to calculate, since the true labels are rarely available in the test data. In this section, we discuss strategies for estimating the test error using only the training data.

In [36]:
import pandas as pd
import numpy as np

#Read the csv file
cars_df = pd.read_csv("car.csv")

#Data cleaning: Remove unwanted columns
cars_df = cars_df.iloc[:,[0,1,3,4,5,6,7,8,10,11,16]]

#Data cleaning: Remove all rows with no sale price
#Why not predict the price and not drop the row? 
#Because rows with no sale price are not guarenteed to have odometer reading
#Additionaly, the dataset is big enough to be able to afford dropping rows
cars_df = cars_df[cars_df["Sale Price"] != 0]

#Data cleaning: Remove all cars that are not Battery Electric Vehicle (Example: Hybrid cars)
cars_df = cars_df[cars_df["Clean Alternative Fuel Vehicle Type"] == "Battery Electric Vehicle (BEV)"]

#Data cleaning: Getting rid of outliers
cars_df = cars_df[cars_df["Odometer Reading"] >= 0]
cars_df = cars_df[cars_df["Odometer Reading"] <= 250000]
cars_df = cars_df[cars_df["Model Year"] >= 2000]
cars_df = cars_df[cars_df["Model Year"] <= 2023]
cars_df = cars_df[cars_df["Sale Price"] <= 300000]
cars_df = cars_df[cars_df["Sale Price"] > 100]

#Since we know each car has a unique vin number, we
#can simplify the vin numbers and use them as index
cars_df["VIN (1-10)"] = cars_df.reset_index().index + 1
cars_df = cars_df.rename(columns={"VIN (1-10)": "Car ID"})

# Split the data into training and test sets.
cars_df = cars_df.set_index("Car ID")
cars_train = cars_df.loc[:100000].copy()
cars_test = cars_df.loc[100000:].copy()

# Smaller sample data set which has chosen random rows (every 10th row)
cars_temp = cars_df.iloc[::10, :]
cars_temp = cars_temp.iloc[:,[1,2,3,6,7,8,9]]

# Log transform the target(Sale Price) for visualization purposes only
cars_train["log(Sale Price)"] = np.log(cars_train["Sale Price"])

cars_df
#cars_train
#cars_test
#cars_temp

,Clean Alternative Fuel Vehicle Type,Model Year,Make,Model,Vehicle Primary Use,Electric Range,Odometer Reading,New or Used Vehicle,Sale Price,Transaction Year
Car ID,,,,,,,,,,
1,Battery Electric Vehicle (BEV),2014,MERCEDES-BENZ,B-Class,Passenger,87,37031,Used,19443,2018
2,Battery Electric Vehicle (BEV),2018,TESLA,Model 3,Passenger,215,50,New,65700,2018
3,Battery Electric Vehicle (BEV),2023,TESLA,Model Y,Passenger,0,15,New,84440,2022
4,Battery Electric Vehicle (BEV),2014,TESLA,Model S,Passenger,208,30840,Used,66864,2017
5,Battery Electric Vehicle (BEV),2022,TESLA,Model Y,Passenger,0,15,New,61440,2022
...,...,...,...,...,...,...,...,...,...,...
109424,Battery Electric Vehicle (BEV),2022,RIVIAN,R1T,Truck,0,100,New,82295,2022
109425,Battery Electric Vehicle (BEV),2022,TESLA,Model 3,Passenger,0,187,New,75290,2022
109426,Battery Electric Vehicle (BEV),2011,NISSAN,Leaf,Passenger,73,64000,Used,1540,2022


## Validation Error

To estimate the test error, we split the training data into a _training set_ and a _validation set_. First, the model is fit to just the data in the training set. Then, the model is evaluated based on its predictions on the validation set. Because the model did not train on any of the labels in the validation set, the validation set essentially plays the role of the test data, even though it was carved out of the training data.

The prediction error on the validation set is known as the _validation error_. The validation error is an approximation to the test error.

To split our data into training and validation sets, we can use the `.sample()` function in `pandas`. Let's use this to split our data into two equal halves, which we will call `train` and `val`.

In [37]:
train = cars_temp.sample(frac=.5)
val = cars_temp.drop(train.index)

#train

Now let's use these training/validation sets to approximate the test MSE of the 5-nearest neighbors model that we fit in Chapter 5.2.

In [53]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor

ct = make_column_transformer(
    (MinMaxScaler(), ["Model Year", "Odometer Reading"]),
    (OneHotEncoder(handle_unknown='ignore'), ["Make", "Model"]),
    remainder="drop"  # all other columns in X will be dropped.
)

pipeline = make_pipeline(
    ct,
    KNeighborsRegressor(n_neighbors=10)
)

pipeline.fit(X=cars_df[["Model Year", "Odometer Reading", "Make", "Model"]], 
             y=cars_df["Sale Price"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  ['Model Year',
                                                   'Odometer Reading']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Make', 'Model'])])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=10))])

We make predictions on the validation set and calculate the validation RMSE:

In [49]:
from sklearn.metrics import mean_squared_error
# extract features and label from validation set
X_val = val[["Model Year", "Odometer Reading", "Make", "Model"]]
y_val = val["Sale Price"]

# get model's predictions on validation set
y_val_ = pipeline.predict(X_val)

# calculate RMSE on validation set
rmse = np.sqrt(mean_squared_error(y_val, y_val_))
rmse

8671.874954144356

Notice that the test error is higher than the training error that we calculated in the previous lesson. In general, this will be true. It is harder for a model to predict for new observations it has not seen, than for observations it has seen!

## Cross Validation

The validation error above was calculated using only 50% of the training data, since we split the training data in half to create the validation set. As a result, the estimate is noisy.

There is a cheap way to obtain a second opinion about how well our model will do on future data. Previously, we split our data at random into two halves, fitting the model to the first half and evaluating it on the second half. Because the model has not already seen the second half of the data, this approximates how well the model would perform on future data. 

But the way we split our data was arbitrary. We might as well swap the roles of the two halves, fitting the model to the _second_ half and evaluating it on the _first_ half. As long as the model is always evaluated on data that is different from the data that was used to train it, we have a valid estimate of test error. A schematic of this approach, known as _cross-validation_, is shown below.

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/cross-validation.png?raw=1)

Because we will be doing all computations twice, just with different data, let's wrap the $k$-nearest neighbors algorithm above into a function called `get_val_error()`, that computes the validation error given training and validation data.

In [40]:
def get_val_error(train, val):
    
    ct = make_column_transformer(
        (MinMaxScaler(), ["Model Year", "Odometer Reading"]),
        (OneHotEncoder(handle_unknown='ignore'), ["Make", "Model"]),
        remainder="drop"  # all other columns in X will be dropped.
    )

    pipeline = make_pipeline(
        ct,
        KNeighborsRegressor(n_neighbors=7)
    )
    
    pipeline.fit(X=cars_temp[["Model Year", "Odometer Reading", "Make", "Model"]], 
             y=cars_temp["Sale Price"])
    
    # extract features and label from validation set
    X_val = val[["Model Year", "Odometer Reading", "Make", "Model"]]
    y_val = val["Sale Price"]
    
    # get model's predictions on validation set
    y_val_ = pipeline.predict(X_val)

    # calculate RMSE on validation set
    rmse = np.sqrt(mean_squared_error(y_val, y_val_))
    
    return rmse

If we apply this function to the training and validation sets from earlier, we get the same estimate of the test error.

In [41]:
get_val_error(train, val)

8618.60474088509

But if we reverse the roles of the training and validation sets, we get another estimate of the test error.

In [42]:
get_val_error(val, train)

8915.055575818576

Now we have two, somewhat independent estimates of the test error. It is common to average the two numbers to obtain an overall estimate of the test error, called the _cross-validation estimate of test error_. Notice that the cross-validation estimate uses each observation in the data exactly once. We make a prediction for each observation, but always using a model that was trained on data that does not include that observation.

## Cross-Validation in scikit-learn

As you know by now, scikit-learn provides functions that automate routine tasks of machine learning. For cross-validation, there is a function, `cross_val_score`, that takes in a model (or pipeline), the training data, and a scoring function, and carries out all aspects of cross-validation, including

1. splitting the training data into training and validation sets
2. fitting the model to each training set
3. calculating the model's predictions on the corresponding validation set
4. calculating the score of the predictions on each validation set.

In [55]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline, 
                         X=cars_df[["Model Year", "Odometer Reading", "Make", "Model"]],
                         y=cars_df["Sale Price"],
                         scoring="neg_mean_squared_error",
                         cv=10)
#scores

First, notice that there are 2 scores. This is because scikit-learn calculated a score from each half of the data when that half served as the validation set.

Second, observe that the scores are negative. This is because scikit-learn requires that a "score" be something that ought to be maximized. Since we want to minimize the mean-squared error, we want to maximize the *negative* mean-squared error. Therefore, the scores that are reported here are the negative of the MSE.

To calculate the RMSE, we negate the negative sign and take a square root.

In [51]:
np.sqrt(-scores)

array([ 8906.72824935, 10210.52099906,  9443.69691343,  9179.9544375 ,
        9279.69911128,  9451.1871284 ,  9007.50765644,  9047.2061077 ,
       10319.94516668,  8983.72589418])

The average of these two scores is the cross-validation estimate of test error.

In [52]:
np.sqrt(-scores).mean()

9383.01716640086

## $K$-Fold Cross Validation

One problem with splitting the training data into two halves is that the model is now trained on only half the amount of data. This model will likely perform worse than the actual model, which is trained on all of the training data. So the cross-validation estimate of test error is unnecessarily pessimistic.

We would like the size of the training set to be closer to the size of the original training data. We can do this by splitting the data into more than two subsamples. In general, we can split the data into $k$ subsamples, alternately training the data on $k-1$ subsamples and evaluating the model on the $1$ remaining subsample, i.e., the validation set. This produces $k$ somewhat independent estimates of the test error. This procedure is known as **$k$-fold cross validation**. (Be careful not to confuse this $k$ with the $k$ in $k$-nearest neighbors.) In hindsight, the  cross validation that we were doing previously is $2$-fold cross validation.

A schematic of $4$-fold cross validation is shown below.

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/k-folds.png?raw=1)

Implementing $k$-fold cross validation in scikit-learn is easy. We simply set the `cv=` parameter to the desired number of folds. For example, the following code carries out $4$-fold cross validation.

In [46]:
scores = cross_val_score(pipeline, 
                         X=cars_temp[["Model Year", "Odometer Reading", "Make", "Model","New or Used Vehicle", "Transaction Year"]],
                         y=cars_temp["Sale Price"],
                         scoring="neg_mean_squared_error",
                         cv=4)
scores

array([-9.55190095e+07, -9.15928569e+07, -9.14499008e+07, -1.04208179e+08])

Notice that $k$ scores are produced, one from each fold. These scores can be averaged to produce a single estimate of the test error.

In [47]:
np.sqrt(-scores).mean()

9778.745830533117